In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
import numpy as np
import pickle
from google.colab.patches import cv2_imshow

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install insightface

In [ ]:
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

In [ ]:
ins_detector = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
ins_detector.prepare(ctx_id=0, det_size=(640, 640))

# Các hàm liên quan insightface

In [ ]:
def get_meta_data(faces):
  data = []
  attributes = ['bbox', 'kps', 'det_score']
  for index in range(len(faces)):
    meta_data = {}
    for attr in attributes:
      meta_data[attr] = faces[index][attr]
    data.append(meta_data)
  return data

In [ ]:
def get_bbox_insightface(faces):
  res_faces = []
  for index in range(len(faces)):
    res = tuple(faces[index]['bbox'])
    res = tuple(map(round, res))
    res = (max(res[1],0), max(res[3],0), max(res[0], 0), max(res[2],0))
    res_faces.append(res)
  return res_faces

In [ ]:
def detect_face_ins(img_path):
  img_path = "".join(img_path.split(".")[:-1])

  img = ins_get_image(img_path)
  faces = ins_detector.get(img)

  #meta_data = get_meta_data(faces)

  res_faces = get_bbox_insightface(faces)
  return res_faces

In [ ]:
def read_img(filepath, mode="RGB"):
    """
    Function to read image from file and convert it to RGB by default, or leaves BGR.
    Input: file path.
    Output: image as numpy array.
    """
    img = cv2.imread(filepath)
    if mode == "RGB":
        # Convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def get_roi(image, coordinates):
    """
    Function to extract region of interest (roi) from image using coordinates.
    Input image: image as numpy array to extract from.
    Input coordinates: coordinates of roi in form of tuple (top, bottom, left, right)
    """
    # Check negative coordinates    
    top = coordinates[0] if coordinates[0] - 5 < 0 else coordinates[0] - 5 
    bottom = coordinates[1] + 5
    left = coordinates[2] if coordinates[2] - 5 < 0 else coordinates[2] - 5
    right = coordinates[3] + 5
    # roi = image[coordinates[0]-5:coordinates[1]+5, coordinates[2]-5:coordinates[3]+5]
    roi = image[top:bottom,left:right]
    return roi

# facenet-pytorch

In [ ]:
!pip install facenet-pytorch
from facenet_pytorch import MTCNN 
model_MTCNN = MTCNN(device='cuda')

In [ ]:
def detect_face_by_facenet_pytorch_mtcnn(model, image_path):  
  #detect_faces_single_image(path_imgg)
  img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
  rows = img.shape[0]
  cols = img.shape[1]
  boxes, probs = model.detect(img, landmarks=False)
  if boxes is None:
    return None
    #print('No face !')
  else:
    for box, prob in zip(boxes, probs):
      x1, y1, x2, y2 = [int(x) for x in box]
      if x1<0: x1=0
      if y1<0: y1=0
      if x2<0: x2=0
      if y2<0: y2=0  
      #print(f"x1={x1}, x2={x2}, y1={y1}, y2={y2}")
      #face = cv2.cvtColor(img[y1:y2, x1:x2],cv2.COLOR_RGB2BGR)
      #cv2_imshow(face)

      if x1 > 4: x1 = x1-5
      else: x1 = 0
      if x2 < cols: x2 = x2 + min(5, cols - x2)
        
      if y1>4: y1=y1-5
      else: y1= 0
      if y2 < rows: y2 = y2 + min(5, rows - y2)      
      #print(f"x1={x1}, x2={x2}, y1={y1}, y2={y2}")
      face = cv2.cvtColor(img[y1:y2, x1:x2],cv2.COLOR_RGB2BGR)
      return face
      #cv2_imshow(face)

# Extract face cho query

In [ ]:
root_dir = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/"
query_dir = os.path.join(root_dir,"query")
faces_query_dir = os.path.join(root_dir,"faces_query")
features_query_dir = os.path.join(root_dir,"features_query/insightface")

list_movies = sorted(os.listdir(query_dir))
for movie_name in list_movies:
  path_to_movie = os.path.join(query_dir,movie_name)
  path_to_movie_faces_query = os.path.join(faces_query_dir,movie_name)
  path_to_features_query = os.path.join(features_query_dir,movie_name)

  if not os.path.isdir(path_to_movie_faces_query):
    os.makedirs(path_to_movie_faces_query)
  if not os.path.isdir(path_to_features_query):
    os.makedirs(path_to_features_query)
  

  list_charactors = sorted(os.listdir(path_to_movie))
  for char_name in list_charactors:
    path_to_movie_char = os.path.join(path_to_movie,char_name)
    path_to_movie_faces_query_char = os.path.join(path_to_movie_faces_query,char_name)
    path_to_features_query_char = os.path.join(path_to_features_query,char_name)
    
    if not os.path.isdir(path_to_movie_faces_query_char):
      os.makedirs(path_to_movie_faces_query_char)
    if not os.path.isdir(path_to_features_query_char):
      os.makedirs(path_to_features_query_char)
      

    list_img_query = sorted(os.listdir(path_to_movie_char))
    for img_query in list_img_query:
      path_to_img_query = os.path.join(path_to_movie_char,img_query)
      #img0 = read_img(path_to_img_query)
      #cv2_imshow(img0)
      # Thưc hiện extract face bằng insightface
      path_to_img_query_by_insightface = "".join(path_to_img_query.split(".")[:-1])
      img = ins_get_image(path_to_img_query_by_insightface)
      faces = ins_detector.get(img)
      if len(faces)>1:
        print(f'insightface: detect {len(faces)} face !', img_query)
      else:
        if faces is None or len(faces)<1 :
          print('insightface: can not detect face !', img_query)
          print('Detecting by MTCNN')
          img_by_MTCNN = cv2.cvtColor(cv2.imread(path_to_img_query), cv2.COLOR_BGR2RGB)
          rows = img_by_MTCNN.shape[0]
          cols = img_by_MTCNN.shape[1]

          boxes, probs, landmarks = model_MTCNN.detect(img_by_MTCNN, landmarks=True)
          if boxes is None or len(boxes)<1 :
            print('MTCNN: can not detect face !', img_query)
          else:
            bbox=boxes[0]
            
            if bbox[0]<0: bbox[0] = 0
            if bbox[1]<0: bbox[1] = 0
            if bbox[2]<0: bbox[2] = 0
            if bbox[3]<0: bbox[3] = 0
            #x1
            if bbox[0]>4: bbox[0] = bbox[0]-5
            else: bbox[0] = 0
            #x2            
            if bbox[2] < cols: bbox[2] = bbox[2] + min(5, cols - bbox[2])
            #y1
            if bbox[1]>4: bbox[1] = bbox[1]-5
            else: bbox[1] = 0
            #y2            
            if bbox[3] < rows: bbox[3] = bbox[3] + min(5, rows - bbox[3])
            #bbox=boxes[0]
            #print(bbox)
            img_face = cv2.cvtColor(img_by_MTCNN[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])],cv2.COLOR_RGB2BGR)
            face = Face(bbox=bbox, kps=landmarks[0], det_score=probs[0])
            embedding = model_recognition.get(img_by_MTCNN, face)
        else:
          res_faces = get_bbox_insightface(faces)
          face = faces[0]
          #print(face)
          img_face = get_roi(img,res_faces[0])
          #img_face = cv2.cvtColor(img_face,cv2.COLOR_RGB2BGR)
          embedding = face['embedding']
        
        framename  = img_query.split(".")[0]
        filename_face = f"{framename}.jpg"

        path_to_img_query_face =  os.path.join(path_to_movie_faces_query_char,filename_face)
        
        cv2_imshow(img_face)
        
        cv2.imwrite(path_to_img_query_face, img_face)
        
        filename_out = f"{framename}.pickle"        
        embedding_file = os.path.join(path_to_features_query_char,filename_out)
        with open(embedding_file, 'wb') as handle:
          pickle.dump(embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(filename_out, ": ", len(embedding))

print("Da hoan thanh viec extract face")

# Deepface Rút trích đăch trưng cho query

In [ ]:
!pip install deepface

In [ ]:
from deepface import DeepFace

In [ ]:
model_name = "Facenet512"
#model_name = "ArcFace"

In [ ]:
root_dir = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/"
faces_query_dir = os.path.join(root_dir,"faces_query")
features_query_dir = os.path.join(root_dir,"features_query/deepface/", model_name)
if not os.path.isdir(features_query_dir):
    os.makedirs(features_query_dir)

query_dir = os.path.join(root_dir,"query")
list_movies = sorted(os.listdir(query_dir))#
for movie_name in list_movies:
  path_to_movie = os.path.join(query_dir,movie_name)
  path_to_movie_faces_query = os.path.join(faces_query_dir,movie_name)
  path_to_features_query = os.path.join(features_query_dir,movie_name)

  if not os.path.isdir(path_to_movie_faces_query):
    print (f'{path_to_movie_faces_query} chưa có')
    continue    
  if not os.path.isdir(path_to_features_query):
    os.makedirs(path_to_features_query)
  

  list_charactors = sorted(os.listdir(path_to_movie))
  for char_name in list_charactors:
    path_to_movie_char = os.path.join(path_to_movie,char_name)
    path_to_movie_faces_query_char = os.path.join(path_to_movie_faces_query,char_name)
    path_to_features_query_char = os.path.join(path_to_features_query,char_name)
    
    if not os.path.isdir(path_to_movie_faces_query_char):
      os.makedirs(path_to_movie_faces_query_char)
    if not os.path.isdir(path_to_features_query_char):
      os.makedirs(path_to_features_query_char)     

    list_img_query = sorted(os.listdir(path_to_movie_char))
    for img_query in list_img_query:
      framename  = img_query.split(".")[0]
      filename_out = f"{framename}.jpg"          
      img_path_face = os.path.join(path_to_movie_faces_query_char,filename_out)
      if os.path.exists(img_path_face):
        img = cv2.imread(img_path_face)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        embedding = DeepFace.represent(img_path = img_path_face, model_name = model_name , enforce_detection=False)
        filename_out = f"{framename}.pickle"
#        embedding = embedding.reshape(-1, 1)
        embedding_file = os.path.join(path_to_features_query_char,filename_out)
        with open(embedding_file, 'wb') as handle:
          pickle.dump(embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)
        cv2_imshow(img)
        print(embedding_file, ": ", len(embedding))

print("Da hoan thanh viec extract feature")